## LLM Integration

Once all the steps were developed:

- Embedding Service
- Ingestion Pipeline
- Context retrieval

Now its time to create the last part of the RAG-LLM technique. Send the context and the user's query to the LLM in order to the LLM to generate an answer.

This time, I'll be using ChatGPT LLM's, but also can work with Google LLM and others.


In [1]:
from google import genai
from google.genai import types
import json
from datetime import datetime

import sys

sys.path.append("..")

from rag_llm_energy_expert.credentials import get_qdrant_config, get_llm_config
from rag_llm_energy_expert.search.searchers import semantic_search
from rag_llm_energy_expert.llm.chat import create_chat_session, generate_response

In [2]:
qdrant_config=get_qdrant_config()
llm_config=get_llm_config()
collection_name = qdrant_config.COLLECTION_NAME + qdrant_config.COLLECTION_VERSION

## Connecting the GENAI client

code from: https://ai.google.dev/gemini-api/docs/text-generation

In [3]:
llm_client = genai.Client(api_key=llm_config.API_KEY.get_secret_value())

Generating multi-turn conversations

The chat format enables users to step incrementally toward answers and to get help with multipart problems.

In [4]:
# Creates a new chat session
chat = llm_client.chats.create(model=llm_config.MODEL)

In [5]:
response = chat.send_message("Hi, im 25 years old")
print(response.text)

Okay! How can I help you today? Are you looking for information about something specific, do you have a question, or are you just looking for conversation? Let me know what you need.



In [6]:
response = chat.send_message("If I am 5 years older than my sister. How old is she?")
response.text

'Since you are 25 and 5 years older than your sister, she is 25 - 5 = 20 years old.\n\nSo your sister is **20** years old.\n'

In [7]:
for message in chat.get_history():
    print(f'role - {message.role}',end=": ")
    print(message.parts[0].text)

role - user: Hi, im 25 years old
role - model: Okay! How can I help you today? Are you looking for information about something specific, do you have a question, or are you just looking for conversation? Let me know what you need.

role - user: If I am 5 years older than my sister. How old is she?
role - model: Since you are 25 and 5 years older than your sister, she is 25 - 5 = 20 years old.

So your sister is **20** years old.



### Configuring parameters

Every prompt sent to the model includes parameters that control how the model generates responses. You can configure these parameters, por let the model use the default options

In [8]:
# Creates a new chat session
chat2 = llm_client.chats.create(
    model=llm_config.MODEL,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0.1
    )
    )

In [9]:
responses = chat2.send_message(
    message = "Hi, If I have 5 apples, and 2 pears, and for all of them I paid 10 USD, if the apples costs 1 USD, how much are the pears?"
)
print(responses.text)

Here's how to solve the problem:

*   **Cost of apples:** 5 apples * $1/apple = $5
*   **Cost of pears:** $10 (total) - $5 (apples) = $5
*   **Cost per pear:** $5 / 2 pears = $2.50/pear

**Answer:** The pears cost $2.50 each.


More model parameters can be found [here](https://ai.google.dev/gemini-api/docs/text-generation)

### System Instructions

System instructions let you steer the behaviour of a model baesd on you specific use case. When you provide system instructions, you give the model additional context to help it understand the task and generate more customized responses. The model should adhere to the system instructions over the full iteraction with the user, enabling you to specify product-level behaviour separete from the prompts provided by end users.

In [10]:
# Creates a new chat session
chat3 = llm_client.chats.create(
    model=llm_config.MODEL,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0.1,
        system_instruction="You are a Mexican energy expert that solves doubts of clients. You must be as direct as possible. Your responses" \
        "shall not be longer than 2 paragraphs (5 lines each)." \
        "The responses shall be based on the context provided. If you don't know the answer, tell that you don't know." \
        "Answer the user's questions in the same language as they're asked."
    )
    )

In [11]:
query = "En el nuevo modelo, cómo se considera a Pemex?"

Semantic Search of the available info in the vector DB

In [12]:

print(semantic_search(
    query=query,
    embedding_model_name=None,
    chunk_overlap=0,
    documents_limit=5,
    collection_name=collection_name
))

2025-04-24 16:42:00.548 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:37 - Preprocessing query...
2025-04-24 16:42:00.555 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:74 - Generating embeddings...
2025-04-24 16:42:05.527 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:87 - Embeddings generated successfully
2025-04-24 16:42:05.527 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:95 - Preparing embeddings for vector search
2025-04-24 16:42:05.540 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:106 - Query preprocessed successfully
2025-04-24 16:42:06.287 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:120 - Processing query results...
2025-04-24 16:42:06.287 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:131 - Query results processed


- Pemex y los particulares podrán reportar a inversionistas y reguladores, así como al público en general, los proyectos que desarrollen en México y los beneficios esperados, siempre
y cuando se afirme en las asignaciones o contratos que el petróleo, el gas natural y los demás hidrocarburos que se encuentren en el subsuelo pertenecen únicamente a México y a los
mexicanos.  
- Se mantendrá y fortalecerá el papel estratégico de Pemex en la industria petrolera. Mediante  
la “Ronda Cero”, Pemex podrá elegir aquellos campos en producción y aquellas áreas en exploración que tengan interés en operar y donde demuestre tener capacidad técnica, financiera y de ejecución para desarrollarlos en forma eficiente y competitiva y podrá migrarlas hacia
un esquema de contratos, con los que podrá acceder a mejores condiciones fiscales.  
- Dada la relevancia que las actividades petroleras revisten para el desarrollo nacional, la reforma constitucional establece que todas las actividades de exploración y

In [13]:
question = "PEMEX podrá migrar a la nueva modalidad de contratación"

response = chat3.send_message(message=question,
                              config = types.GenerateContentConfig(
                                  temperature=0.5,
                                  system_instruction="You are a Mexican energy expert that solves doubts of clients. Your responses" \
        "shall not be longer than 2 paragraphs (5 lines each)." \
        "The responses shall be based on the context provided. If you don't know the answer, tell that you don't know." \
        "Answer the user's questions in the same language as they're asked. Try to generate friendly answers"\
        f"""Context: {semantic_search(query=question,
                                    embedding_model_name=None,
                                    chunk_overlap=0,
                                    collection_name = collection_name,
                                    documents_limit = 5
                                    )}"""
                              ))
print(response.text)

2025-04-24 16:42:06.315 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:37 - Preprocessing query...
2025-04-24 16:42:06.316 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:74 - Generating embeddings...
2025-04-24 16:42:12.392 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:87 - Embeddings generated successfully
2025-04-24 16:42:12.392 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:95 - Preparing embeddings for vector search
2025-04-24 16:42:12.392 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:106 - Query preprocessed successfully
2025-04-24 16:42:13.026 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:120 - Processing query results...
2025-04-24 16:42:13.039 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:131 - Query results processed


¡Hola! Sí, Pemex tiene la opción de migrar a la nueva modalidad de contratación. Esto aplica incluso para contratos que fueron otorgados mediante licitación en el pasado.


## Using the functions generated

In [15]:
chat = create_chat_session(history=[])

2025-04-24 16:42:32.596 | INFO     | rag_llm_energy_expert.llm.chat:create_chat_session:43 - Creating a new chat session...
2025-04-24 16:42:32.598 | INFO     | rag_llm_energy_expert.llm.chat:create_chat_session:53 - Chat session created successfully.


In [16]:
query="PEMEX podrá migrar a la nueva modalidad de contratación"

print(generate_response(prompt=query, chat_session=chat, temperature=0.1))

2025-04-24 16:42:37.642 | INFO     | rag_llm_energy_expert.llm.chat:generate_response:76 - Generating response...
2025-04-24 16:42:37.644 | INFO     | rag_llm_energy_expert.llm.chat:generate_response:77 - Retrieving context...
2025-04-24 16:42:37.645 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:37 - Preprocessing query...
2025-04-24 16:42:37.646 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:74 - Generating embeddings...
2025-04-24 16:42:43.304 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:87 - Embeddings generated successfully
2025-04-24 16:42:43.304 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:95 - Preparing embeddings for vector search
2025-04-24 16:42:43.304 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:106 - Query preprocessed successfully
2025-04-24 16:42:43.723 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_

Pemex podrá migrar a la nueva modalidad de contratación aquellos contratos que en el pasado fueron otorgados por medio de una licitación.



In [ ]:
chat.get_history()

[UserContent(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='PEMEX podrá migrar a la nueva modalidad de contratación')], role='user'),
 Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='Pemex podrá migrar a la nueva modalidad de contratación aquellos contratos que en el pasado fueron otorgados por medio de una licitación.\n')], role='model')]

In [18]:
chat2 = create_chat_session(history = [])

2025-04-24 16:43:18.683 | INFO     | rag_llm_energy_expert.llm.chat:create_chat_session:43 - Creating a new chat session...
2025-04-24 16:43:18.686 | INFO     | rag_llm_energy_expert.llm.chat:create_chat_session:53 - Chat session created successfully.


In [19]:
query = "Cuál es la conclusión de la reforma energética? En qué beneficia?"
llm_response = generate_response(prompt=query, chat_session=chat2, temperature=0.1)
print(llm_response)

2025-04-24 16:43:23.628 | INFO     | rag_llm_energy_expert.llm.chat:generate_response:76 - Generating response...
2025-04-24 16:43:23.630 | INFO     | rag_llm_energy_expert.llm.chat:generate_response:77 - Retrieving context...
2025-04-24 16:43:23.630 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:37 - Preprocessing query...
2025-04-24 16:43:23.631 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:74 - Generating embeddings...
2025-04-24 16:43:29.370 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:87 - Embeddings generated successfully
2025-04-24 16:43:29.372 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:95 - Preparing embeddings for vector search
2025-04-24 16:43:29.372 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:106 - Query preprocessed successfully
2025-04-24 16:43:30.070 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_

La Reforma Energética modernizará la industria energética de México, haciéndola más competitiva e impulsando el desarrollo. Aumentará la renta petrolera del Estado, impulsará el crecimiento económico, creará empleos y fortalecerá a Pemex y a la CFE, para que los mexicanos tengan acceso a energéticos y electricidad más baratos y competitivos.


In [20]:
session_history = []

session_history.append({"role": "user", "parts": [query]})
session_history.append({"role": "model", "parts": [llm_response]})

In [23]:
session_history = [
    {"role": "user", "parts": [{"text": "¿Cuál es la conclusión de la reforma energética?"}]},
    {"role": "model", "parts": [{"text": "La reforma energética busca abrir el sector energético a la inversión privada con el fin de mejorar la eficiencia y la competitividad."}]},
    {"role": "user", "parts": [{"text": "¿Y cómo afecta esto a los consumidores?"}]},
    {"role": "model", "parts": [{"text": "A los consumidores les afecta principalmente en los precios y la calidad del servicio, que podrían mejorar con una mayor competencia."}]}
]


In [ ]:
json.dumps(session_history, ensure_ascii=False)

'[{"role": "user", "parts": ["Cuál es la conclusión de la reforma energética? En qué beneficia?"]}, {"role": "model", "parts": ["La Reforma Energética modernizará la industria energética de México, haciéndola más competitiva e impulsando el desarrollo. Aumentará la renta petrolera del Estado, impulsará el crecimiento económico, creará empleos y fortalecerá a Pemex y a la CFE, para que los mexicanos tengan acceso a energéticos y electricidad más baratos y competitivos."]}]'

In [24]:
chat3 = create_chat_session(history=session_history)

2025-04-24 16:46:28.707 | INFO     | rag_llm_energy_expert.llm.chat:create_chat_session:43 - Creating a new chat session...
2025-04-24 16:46:28.710 | INFO     | rag_llm_energy_expert.llm.chat:create_chat_session:53 - Chat session created successfully.


In [25]:
query = "Me puedes devolver el último query que te hice?"

generate_response(prompt=query, chat_session=chat3)

2025-04-24 16:49:11.806 | INFO     | rag_llm_energy_expert.llm.chat:generate_response:76 - Generating response...
2025-04-24 16:49:11.807 | INFO     | rag_llm_energy_expert.llm.chat:generate_response:77 - Retrieving context...
2025-04-24 16:49:11.809 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:37 - Preprocessing query...
2025-04-24 16:49:11.810 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:74 - Generating embeddings...
2025-04-24 16:49:17.338 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:87 - Embeddings generated successfully
2025-04-24 16:49:17.338 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:95 - Preparing embeddings for vector search
2025-04-24 16:49:17.353 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:106 - Query preprocessed successfully
2025-04-24 16:49:17.839 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_

'Claro, tu última pregunta fue: ¿Y cómo afecta esto a los consumidores?\n'

In [26]:
query = "Me puedes devolver el antepenúltimo query que te hice?"

generate_response(prompt=query, chat_session=chat3)

2025-04-24 16:50:51.085 | INFO     | rag_llm_energy_expert.llm.chat:generate_response:76 - Generating response...
2025-04-24 16:50:51.088 | INFO     | rag_llm_energy_expert.llm.chat:generate_response:77 - Retrieving context...
2025-04-24 16:50:51.089 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:37 - Preprocessing query...
2025-04-24 16:50:51.090 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:74 - Generating embeddings...
2025-04-24 16:50:57.351 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:87 - Embeddings generated successfully
2025-04-24 16:50:57.351 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:95 - Preparing embeddings for vector search
2025-04-24 16:50:57.351 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:106 - Query preprocessed successfully
2025-04-24 16:50:57.784 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_

'Claro, tu antepenúltima pregunta fue: ¿Cuál es la conclusión de la reforma energética?\n'